# Epidemiology of Sedation in Mechanical Ventilation

## Import

In [78]:
import os
os.chdir("..")
os.getcwd()

'/Users/wliao0504/code/clif'

In [79]:
import pandas as pd
import duckdb
from utils import pyCLIF as pc
from utils.waterfall import process_resp_support_waterfall
import ipytest
import tableone

## Cohort Identification

In [80]:
adt = pc.load_data("clif_adt")
hospitalization = pc.load_data("clif_hospitalization")

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_adt.parquet
Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_hospitalization.parquet


### Create ICU-stay level unique id

In [81]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    stitched_encounters = pc.stitch_encounters(hospitalization, adt)

In [82]:
# create a mapping table
query = """
SELECT DISTINCT patient_id, hospitalization_id, encounter_block
FROM stitched_encounters
"""
hosp_to_enc_blk_mapper = duckdb.sql(query).to_df()

In [83]:
query = """
SELECT hospitalization_id
    , encounter_block
    , date_trunc('hour', in_dttm) as in_date_hr
    , 1 as new_icu_stay
FROM stitched_encounters
WHERE location_category = 'icu'
"""
new_icu_start_hours = duckdb.sql(query).to_df()

hosp_ids_w_icu_stays = new_icu_start_hours['hospitalization_id'].unique().tolist()

### Hour 24 and 72

In [84]:
# resp = pc.load_data(
#     table = "clif_respiratory_support",
#     filters = {
#         "hospitalization_id": hosp_ids_w_icu_stays
#     }
#     )

In [85]:
# resp_f = process_resp_support_waterfall(resp)
resp_f = pd.read_parquet("resp_f.parquet")

FileNotFoundError: [Errno 2] No such file or directory: 'resp_f.parquet'

In [ ]:
focal_hosp_ids = [
    '21738444', 
    '20004088', 
    '20006154', 
    '20018306'
    ]

In [ ]:
# query = f"""
# SELECT SUM(CASE WHEN location_category = 'icu' THEN 1 ELSE 0 END) as total_icu_stays
# FROM adt
# WHERE hospitalization_id IN ({",".join(focal_hosp_ids)})
# """
# duckdb.sql(query).to_df()

In [ ]:
resp_f['date_hr'] = resp_f['recorded_dttm'].dt.floor('h')

query = f"""
SELECT t1.hospitalization_id
    , t3.encounter_block
    , t1.date_hr
    , MAX(CASE WHEN t2.new_icu_stay = 1 THEN 1 ELSE 0 END) as new_icu_start_from_adt
    , MAX(CASE WHEN t1.device_category = 'imv' THEN 1 ELSE 0 END) as on_imv
    , MAX(CASE WHEN t1.tracheostomy is True OR t1.tracheostomy = 1 THEN 1 ELSE 0 END) as trach_ever
    , ROW_NUMBER() OVER (PARTITION BY t1.hospitalization_id ORDER BY t1.date_hr) as rn_by_hosp
    , CASE WHEN (
        rn_by_hosp = 1 -- new hospitalization
        OR new_icu_start_from_adt = 1 -- new icu stay
    ) THEN 1 ELSE 0 END as new_icu_stay
FROM resp_f as t1
LEFT JOIN new_icu_start_hours AS t2
    ON t1.hospitalization_id = t2.hospitalization_id
    AND t1.date_hr = t2.in_date_hr
LEFT JOIN hosp_to_enc_blk_mapper AS t3
    ON t1.hospitalization_id = t3.hospitalization_id
-- WHERE t1.hospitalization_id IN ({",".join(focal_hosp_ids)})
GROUP BY t1.hospitalization_id, t1.date_hr, t3.encounter_block
ORDER BY t1.hospitalization_id, t1.date_hr
"""
df1 = duckdb.sql(query).to_df()

In [ ]:
query = """
-- generate unique icu stay ids
WITH t1 AS (
    SELECT hospitalization_id
        , encounter_block
        , date_hr
        , on_imv
        , new_icu_stay
        , SUM(new_icu_stay) OVER (ORDER BY hospitalization_id, date_hr) as icu_stay_id
    FROM df1
    -- keep only hospitalizations that have at least one hour on imv and no tracheostomy
    WHERE hospitalization_id IN (
        SELECT DISTINCT hospitalization_id
        FROM df1
        GROUP BY hospitalization_id
        HAVING MAX(on_imv) = 1 AND MAX(trach_ever) = 0
    )
),
-- generate unique imv streak ids
t2 AS (
    SELECT hospitalization_id
        , icu_stay_id
        -- , encounter_block
        , date_hr
        , on_imv
        , ROW_NUMBER() OVER (PARTITION BY icu_stay_id ORDER BY date_hr) as rn_overall
        , ROW_NUMBER() OVER (PARTITION BY icu_stay_id, on_imv ORDER BY date_hr) as rn_by_imv_status
        , rn_overall - rn_by_imv_status as imv_streak_id
    FROM t1
    -- keep only icu stays that have at least one hour on imv
    WHERE icu_stay_id IN (
        SELECT DISTINCT icu_stay_id
        FROM t1
        GROUP BY icu_stay_id
        HAVING MAX(on_imv) = 1
    )
    ORDER BY hospitalization_id, icu_stay_id, date_hr
),
-- mark the 24th and 72th hour of each imv streak
t3 AS (
    SELECT hospitalization_id, icu_stay_id, date_hr
        , imv_streak_id, on_imv
        , SUM(on_imv) OVER (PARTITION BY icu_stay_id, imv_streak_id ORDER BY date_hr) as imv_hrs_in_streak
        , CASE WHEN (imv_hrs_in_streak = 24) THEN 1 ELSE 0 END as hr_24_on_imv
        , CASE WHEN (imv_hrs_in_streak = 72) THEN 1 ELSE 0 END as hr_72_on_imv
        -- calculate hour since first intubation within each icu stay
        , MIN(CASE WHEN on_imv = 1 THEN date_hr END) OVER (PARTITION BY icu_stay_id) as first_imv_hr_in_icu_stay
        -- can only calculate diff in secs, so convert to hrs
        ,  EXTRACT(EPOCH FROM (date_hr - first_imv_hr_in_icu_stay)) / 3600 + 1 as hrs_since_first_imv
    FROM t2
    ORDER BY hospitalization_id, icu_stay_id, date_hr
    )
-- exclude cases with reintubation within 72 hours
SELECT hospitalization_id, icu_stay_id, date_hr
    , imv_streak_id, on_imv, imv_hrs_in_streak, hrs_since_first_imv
    , hr_24_on_imv, hr_72_on_imv
    , COUNT(DISTINCT CASE WHEN hrs_since_first_imv BETWEEN 0 AND 72 THEN imv_streak_id END) 
        OVER (PARTITION BY icu_stay_id) as n_imv_streaks_in_72_hrs
    , CASE WHEN n_imv_streaks_in_72_hrs <= 2 AND hr_24_on_imv = 1 THEN 1 ELSE 0 END as hr_24_on_imv_noreintub
    , CASE WHEN n_imv_streaks_in_72_hrs = 1 AND hr_72_on_imv = 1 THEN 1 ELSE 0 END as hr_72_on_imv_noreintub
FROM t3
ORDER BY hospitalization_id, icu_stay_id, date_hr
"""
df2 = duckdb.sql(query).to_df()

In [ ]:
%%ipytest

# sanity tests against the MIMIC-IV data
@pytest.mark.parametrize("hospitalization_id,date_hr,expected_hr,expected_flag,expected_flag_noreintub", [
    # on imv for 24-hrs twice during the same hospitalization -- so would be excluded if no reintubation within 72 hrs
    (21738444, "2186-09-14 17:00:00-06:00", 24, 1, 0),  
    (21738444, "2186-09-14 18:00:00-06:00", 24, 0, 0),  
    (21738444, "2186-09-16 18:00:00-06:00", 24, 1, 0), # second streak within the hosp
    # not on imv for the first few hrs but long streak afterwards
    (20004088, "2159-09-30 09:00:00-06:00", 24, 1, 1),
    (20004088, "2159-10-02 09:00:00-06:00", 72, 1, 1),
    # very short streaks: 20006154
    # 3 icu stays within the same hospitalization
    (20018306, "2136-05-16 05:00:00-06:00", 24, 1, 1),
    (20018306, "2136-07-01 19:00:00-06:00", 24, 1, 1),
    # (20018306, "2136-06-01 03:00:00-06:00", 24, 0), # in a icu stay that was filtered out in the df because of no imv ever
])
def test_if_on_imv_at_hr_x(hospitalization_id, date_hr, expected_hr, expected_flag, expected_flag_noreintub):
    query = f"""
    SELECT hr_{expected_hr}_on_imv, hr_{expected_hr}_on_imv_noreintub
    FROM df2
    WHERE hospitalization_id = {hospitalization_id}
    AND date_hr = '{date_hr}'
    """
    result = duckdb.sql(query).to_df()
    observed_flag = result[f'hr_{expected_hr}_on_imv'].iloc[0]
    observed_flag_noreintub = result[f'hr_{expected_hr}_on_imv_noreintub'].iloc[0]
    assert observed_flag == expected_flag
    assert observed_flag_noreintub == expected_flag_noreintub

UsageError: Cell magic `%%ipytest` not found.


### The Cohort

In [ ]:
# keep the cohort
query = """
SELECT hospitalization_id
    , encounter_block
    --, icu_stay_id
    , date_hr
    , CASE WHEN hr_24_on_imv_noreintub = 1 THEN 'hr_24'
        WHEN hr_72_on_imv_noreintub = 1 THEN 'hr_72'
        ELSE NULL END as cohort_flag
FROM df2
LEFT JOIN hosp_to_enc_blk_mapper USING (hospitalization_id)
WHERE hr_24_on_imv_noreintub = 1 OR hr_72_on_imv_noreintub = 1
"""
cohort = duckdb.sql(query).to_df()

cohort_hosp_ids = cohort['hospitalization_id'].unique().tolist()

In [ ]:
query = """
SELECT *
    , 
FROM cohort
WHERE cohort_flag IS NOT NULL AND med_category is NOT NULL
"""
mac_cohort_hrs = duckdb.sql(query).to_df()

## Vitals

In [142]:
vital_categories = [
    "weight_kg",
    "height_cm",
    "temp_c"
    "heart_rate",
    "respiratory_rate",
    "temperature",
    "sbp",
    "dbp",
    "spo2",
    "map"
]

In [143]:
query = f"""
SELECT *,
    UNNEST(vital_categories) as vital_category
FROM cohort
"""
duckdb.sql(query).to_df()

BinderException: Binder Error: Referenced column "vital_categories" not found in FROM clause!
Candidate bindings: "hospitalization_id", "date_hr", "cohort_flag"

In [ ]:
vitals_required_columns = [
    "hospitalization_id",
    "recorded_dttm",
    "vital_category",
    "vital_value"
]

vitals = pc.load_data(
    table = "clif_vitals",
    filters = {
        "hospitalization_id": cohort_hosp_ids
    },
    columns = vitals_required_columns
)

vitals['date_hr'] = vitals['recorded_dttm'].dt.floor('h')

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_vitals.parquet


In [ ]:
query = """
-- fill any missing values in the cohort hours with the nearest in time (in the past or future)
SELECT hospitalization_id
    , date_hr
    , cohort_flag
    , vital_category
    , MEAN(vital_value) as mean_value
    /*
    , LAG(mean_value) OVER (PARTITION BY hospitalization_id, date_hr, vital_category ORDER BY date_hr) as mean_value_last
    , LAG(date_hr) OVER (PARTITION BY hospitalization_id, vital_category ORDER BY date_hr) as date_hr_last
    , (date_hr - date_hr_last) as distance_to_last
    , LEAD(mean_value) OVER (PARTITION BY hospitalization_id, date_hr, vital_category ORDER BY date_hr) as mean_value_next
    , LEAD(date_hr) OVER (PARTITION BY hospitalization_id, vital_category ORDER BY date_hr) as date_hr_next
    , (date_hr_next - date_hr) as distance_to_next
    , CASE WHEN distance_to_last < distance_to_next THEN mean_value_last ELSE mean_value_next END as mean_value_nearest
    , COALESCE(mean_value, mean_value_nearest) as mean_value_final
    */
FROM cohort
FULL OUTER JOIN vitals USING (hospitalization_id, date_hr)
GROUP BY hospitalization_id, date_hr, cohort_flag, vital_category
ORDER BY hospitalization_id, vital_category, date_hr, cohort_flag
"""
vitals_hrly = duckdb.sql(query).to_df()

In [ ]:
query = """
SELECT hospitalization_id
    , date_hr
    , cohort_flag
    , vital_category
    , mean_value
    , mean_value_final
FROM vitals_hrly
WHERE cohort_flag IS NOT NULL
ORDER BY hospitalization_id, vital_category, date_hr, cohort_flag
"""
vitals_cohort_hrs = duckdb.sql(query).to_df()

## Medications

In [86]:
sed_med_categories = [
    "midazolam", "lorazepam", "hydromorphone", "fentanyl", "propofol", "dexmedetomidine", "ketamine"
]

mac_required_columns = [
    "hospitalization_id", 
    "admin_dttm",
    "med_category",
    "med_dose",
    "med_dose_unit",
    "mar_action_name"
]

mac = pc.load_data(
    table = "clif_medication_admin_continuous",
    columns = mac_required_columns,
    filters = {
        "hospitalization_id": cohort_hosp_ids,
        "med_category": sed_med_categories
    }
)

mac['date_hr'] = mac['admin_dttm'].dt.floor('h')

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_medication_admin_continuous.parquet


### Check dosage unit

In [87]:
query = """
SELECT med_category, med_dose_unit, COUNT(*) as n
FROM mac
GROUP BY med_category, med_dose_unit
ORDER BY med_category, n DESC
"""
med_units_count = duckdb.sql(query).to_df()
med_units_count

,med_category,med_dose_unit,n
0,dexmedetomidine,mcg/kg/hour,89430
1,fentanyl,mcg/hour,113807
2,fentanyl,mcg/kg/hour,2
3,hydromorphone,mg/hour,9356
4,ketamine,mg/kg/hour,7513
5,ketamine,mg/hour,101
6,ketamine,mcg/kg/min,33
7,ketamine,mcg/min,11
8,lorazepam,mg/hour,128
9,midazolam,mg/hour,46571


In [137]:
query = """
SELECT c.hospitalization_id
    , c.cohort_flag
    , med_category
    , c.date_hr
    , LAG(admin_dttm) OVER (PARTITION BY c.hospitalization_id, med_category ORDER BY admin_dttm) as admin_dttm_last
    , LAG(med_dose) OVER (PARTITION BY c.hospitalization_id, med_category ORDER BY admin_dttm) as med_dose_last
    , admin_dttm
    , med_dose
    , med_dose_unit
    , mar_action_name
    , mean_value_final as weight_kg
FROM cohort c
LEFT JOIN vitals_cohort_hrs v 
    ON c.hospitalization_id = v.hospitalization_id
    AND c.date_hr = v.date_hr
    AND v.vital_category = 'weight_kg'
FULL OUTER JOIN mac m 
    ON c.hospitalization_id = m.hospitalization_id
    AND c.date_hr = m.date_hr
ORDER BY c.hospitalization_id, c.date_hr, admin_dttm, med_category, c.cohort_flag
"""
mac_hrly = duckdb.sql(query).to_df()

In [138]:
query = """
SELECT *
    , CASE WHEN admin_dttm = MIN(admin_dttm) OVER (PARTITION BY hospitalization_id, med_category, date_hr) 
        THEN 1 ELSE 0 END as is_first_streak
    , CASE WHEN admin_dttm = MAX(admin_dttm) OVER (PARTITION BY hospitalization_id, med_category, date_hr) 
        THEN 1 ELSE 0 END as is_last_streak
    , date_hr + INTERVAL '1 hour' as date_hr_next
FROM mac_hrly
WHERE cohort_flag IS NOT NULL AND med_category is NOT NULL
ORDER BY hospitalization_id, med_category, date_hr, admin_dttm
"""
mac_cohort_hrs = duckdb.sql(query).to_df()

In [115]:
query = """
SELECT hospitalization_id, cohort_flag, med_category, date_hr
    , SUM(CASE 
        WHEN is_first_streak != 1 
            THEN EXTRACT(EPOCH FROM (admin_dttm - admin_dttm_last))/60.0 * med_dose_last 
        WHEN is_first_streak = 1
        THEN EXTRACT(EPOCH FROM (admin_dttm - date_hr))/60.0 * med_dose_last 
        WHEN is_last_streak = 1
            THEN EXTRACT(EPOCH FROM (date_hr_next - admin_dttm))/60.0 * med_dose_last
        ELSE 0 END) as total_dosage
FROM mac_cohort_hrs
GROUP BY hospitalization_id, med_category, date_hr, cohort_flag
"""
mac_cohort_dosage = duckdb.sql(query).to_df()